In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Maza\\Desktop\\job_aplication_agent'

In [3]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Union

@dataclass(frozen=True)
class CVConfig:
    CV_file: Union[Path, str]
    edited_CV: Union[Path, str]
    job_description: Union[Path, str]
    qualifications: Union[Path, str]
    contact_info: Union[Path, str]
    about_me: Union[Path, str]
    education: Union[Path, str]
    experience: Union[Path, str]
    model: str
    CV_prompt_templates: dict

    def __post_init__(self):
        # Convert string paths to Path objects
        object.__setattr__(self, 'CV_file', Path(self.CV_file))
        object.__setattr__(self, 'edited_CV', Path(self.edited_CV))
        object.__setattr__(self, 'job_description', Path(self.job_description))
        object.__setattr__(self, 'qualifications', Path(self.qualifications))
        object.__setattr__(self, 'contact_info', Path(self.contact_info))
        object.__setattr__(self, 'about_me', Path(self.about_me))
        object.__setattr__(self, 'education', Path(self.education))
        object.__setattr__(self, 'experience', Path(self.experience))


In [4]:
from dotenv import load_dotenv
from pathlib import Path


from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

from job_aplication_agent.utils.common import  load_json,read_yaml,read_file,save_file
from job_aplication_agent import logger
from job_aplication_agent.constants import *
load_dotenv()

True

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        models_filepath = MODELS_FILE_PATH,
        prompt_template = PROMPT_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.models = read_yaml(models_filepath)
        self.prompt_template = read_yaml(prompt_template)
    
        

    
    def get_CV_config(self) -> CVConfig:
        """
        
        Returns:
            
        """
        
        CV_config = self.config.CV
        prompt_templates = self.prompt_template
        
        create_CV_config = CVConfig(
            CV_file= CV_config.CV_file,
            edited_CV = CV_config.edited_CV,
            job_description= CV_config.job_description,
            qualifications= CV_config.qualifications,
            contact_info=CV_config.contact_info,
            about_me=CV_config.about_me,
            education=CV_config.education,
            experience=CV_config.experience,
            model= self.models.Llama3,
            CV_prompt_templates = prompt_templates.cv_prompt_templates

           
        ) 

        return create_CV_config

In [18]:
class GetCV:
    def __init__(self,config:CVConfig):
        self.config = config
        self.CV_prompt_templates = config.CV_prompt_templates
        try:
            self.CV = read_file(config.CV_file)
            logger.info(f"CV loaded")
        except Exception as e:
            logger.error(f"Loading CV error {e}")

        try:
            self.qualifications = read_file(config.qualifications)
            logger.info(f"Qualifications loaded")
        except Exception as e:
            logger.error(f"Loading qualifications error {e}")

        try:
            self.job_description = read_file(config.job_description)
            logger.info(f"Job description loaded")
        except Exception as e:
            logger.error(f"Loading job description error {e}")

    def create_CV_agents(self):
        llm = ChatGroq(temperature=0,model_name= self.config.model)
        logger.info(f"Llm initialized")

        #Agents
        try:
            contact_info_prompt = PromptTemplate(
                template= self.CV_prompt_templates.contact_info_template,
                input_variables=["CONTACT_INFO","CV"]
            )
            self.contact_info_agent = contact_info_prompt | llm | StrOutputParser()
            logger.info("Contact info Agent created")
        except Exception as e:
            logger.error("Creating contact info agent error: {e}")

        try:
            summary_prompt = PromptTemplate(
                template= self.CV_prompt_templates.summary_template,
                input_variables=["JOB_DESCRIPTION","QUALIFICATIONS","CV"]
            )
            self.summary_agent = summary_prompt | llm | StrOutputParser()
            logger.info("Summary Agent created")
        except Exception as e:
            logger.error("Creating summary agent error: {e}")

        try:
            skills_prompt = PromptTemplate(
                template= self.CV_prompt_templates.skills_template,
                input_variables=["JOB_DESCRIPTION","QUALIFICATIONS","CV"]
            )
            self.skills_agent = skills_prompt | llm | StrOutputParser()
            logger.info("Skills Agent created")
        except Exception as e:
            logger.error("Creating skills agent error: {e}")

    def get_contact_info(self):
        logger.info(f"Updating contact info")
        try:
            contact_info = read_yaml(self.config.contact_info)
            logger.info(f"Contact info loaded")
        except Exception as e:
            logger.error(f"Loading contact info error {e}")
        
        try:
            cv_conatct_info = self.contact_info_agent.invoke({
            "CONTACT_INFO":contact_info,
            "CV":self.CV
            })
            self.CV = cv_conatct_info
            save_file(self.config.edited_CV,cv_conatct_info)
            logger.info(f"Contact info updated")
        except Exception as e:
            logger.error(f"Updating contact info error {e}")
    
    def get_summary(self):
        logger.info(f"Updating summary")
        try:
            cv_summary = self.summary_agent.invoke({
                "JOB_DESCRIPTION":self.job_description,
                "QUALIFICATIONS":self.qualifications,
                "CV":self.CV
                })
            self.CV = cv_summary
            save_file(self.config.edited_CV,cv_summary)
            logger.info(f"Summary updated")
        except Exception as e:
            logger.error(f"Updating summary error {e}")
    
    def get_skills(self):
        logger.info(f"Updating skills")
        try:
            cv_skills = self.skills_agent.invoke({
                "JOB_DESCRIPTION":self.job_description,
                "QUALIFICATIONS":self.qualifications,
                "CV":self.CV
                })
            self.CV = cv_skills
            save_file(self.config.edited_CV,cv_skills)
            logger.info(f"Skills updated")
        except Exception as e:
            logger.error(f"Updating skills error {e}")


In [19]:
config = ConfigurationManager()
cv_config = config.get_CV_config()
get_cv = GetCV(cv_config)
get_cv.create_CV_agents()
contact_info_cv = get_cv.get_contact_info()
cv_summary = get_cv.get_summary()
cv_summary = get_cv.get_skills()

[2024-07-09 15:29:45,379: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-07-09 15:29:45,382: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-07-09 15:29:45,400: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
[2024-07-09 15:29:45,401: INFO: 1414791549: CV loaded:]
[2024-07-09 15:29:45,403: INFO: 1414791549: Qualifications loaded:]
[2024-07-09 15:29:45,404: INFO: 1414791549: Job description loaded:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Llm initialized:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Contact info Agent created:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Summary Agent created:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Skills Agent created:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Updating contact info:]
[2024-07-09 15:29:45,898: INFO: common: yaml file: prompt_text_inputs\contact_info.yaml loaded successfully:]
[2024-07-09 15:29:45,898: INFO: 1414791549: Contact info loaded:]
[2024-07-09 15:29:51,7